In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import precision_recall_curve, roc_curve

from sklearn.preprocessing import label_binarize

from sklearn.preprocessing import MultiLabelBinarizer

from numpy import sqrt

from numpy import argmax
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import metrics



from sklearn.metrics import precision_score, recall_score, accuracy_score

from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,roc_curve
import sklearn.feature_extraction.text as text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import cross_validate

from sklearn.model_selection import GridSearchCV


In [2]:
data = pd.read_csv("final_data_rp2_p10.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.describe(include="all")

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,TABLE_NAME,COLUMN_NAME,irm_table_field_description_tag.ddtext,irm_table_field_description_tag.tag_name
count,20184,19921,20240,20203,20254,20394,20096,20259,20486,20265,...,19877,19993,20031,19890,19823,19916,44535,44535,44535,2846
unique,2037,2003,2052,2036,2094,2070,2039,2060,2083,2057,...,2104,2066,2074,2052,2005,2099,206,11954,6673,5
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,EKPO,EKPO.OIDATFM1,Client,RIL Confidential_BSFD
freq,4332,4309,4371,4316,4314,4392,4340,4299,4296,4297,...,4325,4354,4347,4340,4410,4320,2271,6,726,2183


In [3]:
data = data[['irm_table_field_description_tag.ddtext','irm_table_field_description_tag.tag_name']]

In [4]:
data.shape

(44535, 2)

In [5]:
data_cls = data.fillna('Others')

In [6]:
data_cls.shape

(44535, 2)

In [7]:
#data1 = data1[data1['rp2_table_field_description_tag.irm_tag']!="RIL Confidential_SPDI"]
data_cls

,irm_table_field_description_tag.ddtext,irm_table_field_description_tag.tag_name
0,Client,Others
1,Client,Others
2,Address number,Others
3,Address number,Others
4,Person number,Others
...,...,...
44530,Date of Last Change,Others
44531,Date of Last Change,Others
44532,Last changed at,Others
44533,Last changed at,Others


In [8]:
data_cls_new = data_cls.reset_index().drop_duplicates(subset=['irm_table_field_description_tag.ddtext','irm_table_field_description_tag.tag_name'],keep='first').set_index('index')
data_cls_new=data_cls_new.loc[(data_cls_new["irm_table_field_description_tag.tag_name"]!="RIL Confidential_SPCI") & (data_cls_new["irm_table_field_description_tag.tag_name"]!="RIL Confidential_BSSD")]
#data_cls_new.to_csv("Data_distribution.cv")


print(data_cls_new.shape)
grouped= data_cls_new.groupby('irm_table_field_description_tag.tag_name')
Tag_count=pd.DataFrame()
Tag_count["Number_of_tags"]=grouped['irm_table_field_description_tag.tag_name'].size()
Tag_count

(7317, 2)


,Number_of_tags
irm_table_field_description_tag.tag_name,
Others,6523
RIL Confidential_BSFD,436
RIL Confidential_BSTD,247
RIL Confidential_SPDI,111


In [9]:
data_cls_new_1=data_cls_new.loc[data_cls_new["irm_table_field_description_tag.tag_name"]=="Others"]
data_cls_new_1 = data_cls_new_1.loc[np.random.choice(data_cls_new_1.index, size=2800)]
#data_cls_new_1

data_cls_new=data_cls_new.loc[data_cls_new["irm_table_field_description_tag.tag_name"]!="Others"]
data_cls_new=pd.concat([data_cls_new_1,data_cls_new])
data_cls_new

,irm_table_field_description_tag.ddtext,irm_table_field_description_tag.tag_name
index,,
18517,Invoice Document Number of Copied Invoice,Others
27064,Accounting Document-2 (Payable),Others
43440,Invoice Base amount,Others
15144,Purchasing Across Purchasing Group,Others
7696,Item number of allocation table,Others
...,...,...
43771,margin amount,RIL Confidential_BSFD
43825,Total Funding Amount,RIL Confidential_BSFD
43836,Vendor Account Number,RIL Confidential_BSFD


Binary Classification (Others Vs All tags)

In [10]:
corpus=data_cls_new['irm_table_field_description_tag.ddtext'].tolist()

In [11]:
tfidf=text.TfidfVectorizer(corpus,stop_words="english",max_features=5000,min_df=2, max_df=0.2)  #removes stopwords

In [12]:
tfidf_matrix=tfidf.fit_transform(corpus)

In [13]:
tfidf_matrix

<3594x1095 sparse matrix of type '<class 'numpy.float64'>'
	with 11244 stored elements in Compressed Sparse Row format>

In [14]:
data2 = pd.DataFrame(tfidf_matrix.toarray(),columns=tfidf.get_feature_names())

In [15]:
data2

,00010101,10,195,1st,20,2nd,3rd,4th,50,6b,...,wm,wms,word,work,workflow,works,year,yyyymmddhhmmss,zip,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Save vectorizer.vocabulary_
pickle.dump(tfidf.vocabulary_,open("TFIDF_feature.pkl","wb"))

In [17]:
# Import label encoder 
data_all_tags=data2.copy()
#data_2_tags=data2.copy()

from sklearn import preprocessing 

  

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 



# Encode labels in column 'species'. 

data_all_tags['Target']= label_encoder.fit_transform(data_cls_new['irm_table_field_description_tag.tag_name'])
#data_2_tags['Target']= label_encoder.fit_transform(data_binary_df['irm_table_field_description_tag.tag_name'])


In [18]:
data_all_tags["Target"].value_counts() #BSFD - 1, Others - 0
#data_2_tags["Target"].value_counts()

0    2800
1     436
2     247
3     111
Name: Target, dtype: int64

Data Preparation for 3 tags model(Second stage Model)

In [19]:
data_3_tags=data_all_tags.loc[data_all_tags["Target"]!=0]
data_3_tags

,00010101,10,195,1st,20,2nd,3rd,4th,50,6b,...,wms,word,work,workflow,works,year,yyyymmddhhmmss,zip,zone,Target
2800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


Binary Modeling start(First Stage Model)

In [20]:
# split data into X and y
X = data_all_tags.iloc[:,:-1]
Y = data_all_tags.iloc[:,-1]


In [21]:
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X,Y,random_state = 42, test_size=0.3, stratify=Y )

In [22]:
y_test_df.value_counts()

0    841
1    131
2     74
3     33
Name: Target, dtype: int64

In [23]:
y_train_binary=y_train_df.replace([2,3], 1)
y_test_binary=y_test_df.replace([2,3], 1)

In [24]:
y_test_binary.value_counts()

0    841
1    238
Name: Target, dtype: int64

In [25]:
model1 = XGBClassifier()
model1.fit(X_train_df,y_train_binary)
y_pred = model1.predict(X_test_df)


In [26]:
pickle.dump(model1,open("Binary_Xgboost.pkl","wb"))

In [27]:
from numpy import sqrt
from numpy import argmax


proba1 = model1.predict_proba(X_test_df)[:,1]
#probs = model.predict_proba(testX)
    
# proba1 = probability_df[1]
#print(proba1)


#Y_pred_threshold = (proba >= 0.918760).astype(bool)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_test_binary, proba1)
# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

Best Threshold=0.152638, G-Mean=0.696


In [28]:
Y_pred_threshold = (proba1 >=thresholds[ix]).astype(bool)

In [29]:
print("Precision = {}".format(precision_score(y_test_binary, Y_pred_threshold,average='macro')))
print("Recall = {}".format(recall_score(y_test_binary, Y_pred_threshold,average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test_binary, Y_pred_threshold)))
auc = roc_auc_score(y_test_binary, Y_pred_threshold)
print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test_binary,Y_pred_threshold)
print(cm)

Precision = 0.6521353746978243
Recall = 0.6980835140239211
Accuracy = 0.726598702502317
AUC: 0.698
[[630 211]
 [ 84 154]]


Multilabel XGBOOST on 3 tags

In [30]:
X_test_df['Prediction']=Y_pred_threshold
X_test_temp=X_test_df.loc[X_test_df["Prediction"]==True]
X_test_temp_fn=X_test_df.loc[X_test_df["Prediction"]==False]
X_test_temp_fn

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,00010101,10,195,1st,20,2nd,3rd,4th,50,6b,...,wms,word,work,workflow,works,year,yyyymmddhhmmss,zip,zone,Prediction
1162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [31]:
result = X_test_temp.join(data_all_tags["Target"], how='inner')
test_3_tags_df_tp=result.loc[result["Target"]!=0]

result_fn = X_test_temp_fn.join(data_all_tags["Target"], how='inner')
test_3_tags_df_fn=result_fn.loc[result_fn["Target"]!=0]
val_3_tags_df=pd.concat([test_3_tags_df_tp,test_3_tags_df_fn])
data_fp=result.loc[result["Target"]==0]

val_3_tags_df=val_3_tags_df.drop(columns=["Prediction"])
data_fp=data_fp.drop(columns=["Prediction"])
val_3_tags_df


,00010101,10,195,1st,20,2nd,3rd,4th,50,6b,...,wms,word,work,workflow,works,year,yyyymmddhhmmss,zip,zone,Target
3515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3
3024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
2867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3
3022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
2937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,2
3385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1
3324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,2
2948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1


In [32]:
data_complete_3_tags= data_3_tags[~data_3_tags.index.isin(val_3_tags_df.index)]

#X_3_tags = data_complete_3_tags.iloc[:,:-1]
#Y_3_tags = data_complete_3_tags.iloc[:,-1]
#Y_3_tags
#X_3_tags_train, X_3_tags_test, Y_3_tags_train, Y_3_tags_test = train_test_split(X_3_tags,Y_3_tags,random_state = 42, test_size=0.3, stratify=Y )
#vr=int(data_3_tags.shape[0]*0.3-test_3_tags_df.shape[0])
#print(train_3_tags_temp["Target"].value_counts())
train_3_tags, test_3_tags= train_test_split(data_complete_3_tags, test_size=0.3,stratify=data_complete_3_tags["Target"],random_state=10)
#test_3_tags_df=pd.concat([test_3_tags_df,test_3_tags_df_temp])
#train_3_tags_df
#test_3_tags_df
data_complete_3_tags["Target"].value_counts()

1    305
2    173
3     78
Name: Target, dtype: int64

In [33]:
X_3_tags_train=train_3_tags.iloc[:,:-1]
Y_3_tags_train=train_3_tags.iloc[:,-1]-1
X_3_tags_test = test_3_tags.iloc[:,:-1]
Y_3_tags_test = test_3_tags.iloc[:,-1]-1

In [34]:
Y_3_tags_test

3459    0
2804    2
3127    1
3161    1
3191    0
       ..
2979    0
2917    0
2857    0
3023    0
3082    1
Name: Target, Length: 167, dtype: int64

In [35]:
model2 = XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0.0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.2, max_delta_step=0, max_depth=7,
       min_child_weight=1, monotone_constraints=None,
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='multi:softmax', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=None)
#model1=XGBClassifier()
model2.fit(X_3_tags_train, Y_3_tags_train)
y_pred_2 = model2.predict(X_3_tags_test)

In [38]:
y_pred_2

array([0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 1,
       0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1,
       2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 1, 2, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 1, 1, 0, 1, 0, 0, 2, 0, 2, 1, 0, 2, 1, 0, 0, 0, 2,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [39]:
pickle.dump(model2,open("Multi_Xgboost.pkl","wb"))

In [36]:
print("Precision = {}".format(precision_score(Y_3_tags_test, y_pred_2, average='macro')))
print("Recall = {}".format(recall_score(Y_3_tags_test, y_pred_2, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_3_tags_test, y_pred_2)))
#auc = roc_auc_score(Y_test, y_pred)
#print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(Y_3_tags_test,y_pred_2)
print(cm)

Precision = 0.8277224593014066
Recall = 0.7513935340022296
Accuracy = 0.8203592814371258
[[89  3  0]
 [15 33  4]
 [ 7  1 15]]


In [ ]:
x_val=val_3_tags_df.iloc[:,:-1]
y_val=val_3_tags_df.iloc[:,-1]-1
y_pred_val = model2.predict(x_val)

In [ ]:
print("Precision = {}".format(precision_score(y_val, y_pred_val, average='macro')))
print("Recall = {}".format(recall_score(y_val, y_pred_val, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_val, y_pred_val)))
#auc = roc_auc_score(Y_test, y_pred)
#print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_val,y_pred_val)
print(cm)

In [ ]:
test_3_tags_df_tp=test_3_tags_df_tp.drop(columns=["Prediction"])
x_tp=test_3_tags_df_tp.iloc[:,:-1]
y_tp=test_3_tags_df_tp.iloc[:,-1]-1
y_pred_tp = model2.predict(x_tp)

In [ ]:
print("Precision = {}".format(precision_score(y_tp, y_pred_tp, average='macro')))
print("Recall = {}".format(recall_score(y_tp, y_pred_tp, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_tp, y_pred_tp)))
#auc = roc_auc_score(Y_test, y_pred)
#print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_tp,y_pred_tp)
print(cm)

In [ ]:
y_pred_fp=model2.predict(data_fp.iloc[:,:-1])
pd.DataFrame(y_pred_fp).to_csv("Multilabel_FP.csv")
pd.DataFrame(y_pred_fp)[0].value_counts()

ONE VS REST CLASSIFIER ON 3 TAGS

In [ ]:
X_train_ovr=X_3_tags_train.to_numpy()
X_test_ovr=X_3_tags_test.to_numpy()
y_train_ovr=Y_3_tags_train.to_numpy()
y_test_ovr=Y_3_tags_test.to_numpy()
x_val_ovr=x_val.to_numpy()
y_val_ovr=y_val.to_numpy()
x_tp=x_tp.to_numpy()
y_tp=y_tp.to_numpy()

In [ ]:
y_train_ls=y_train_ovr.astype(int).tolist()
y_train_one = label_binarize(y_train_ls, classes=[0,1,2])

y_test_ls=y_test_ovr.astype(int).tolist()
y_test_one = label_binarize(y_test_ls, classes=[0, 1, 2])


y_val_ls=y_val_ovr.astype(int).tolist()
y_val_one = label_binarize(y_val_ls, classes=[0, 1, 2])

y_tp_ls=y_tp.astype(int).tolist()
y_tp_one = label_binarize(y_tp_ls, classes=[0, 1, 2])


In [ ]:

#labels have to be modified

n_classes = 3
clf = OneVsRestClassifier(XGBClassifier(n_estimators=50, max_depth=3,random_state=0))
clf.fit(X_train_ovr, y_train_one)
proba1 = clf.predict_proba(X_test_ovr)
y_score = clf.predict_proba(X_test_ovr)
fpr = dict()
tpr = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_one[:, i], y_score[:, i])
    plt.plot(fpr[i], tpr[i], lw=2, label='class {}'.format(i))
    gmeans = sqrt(tpr[i] * (1-fpr[i]))
    ix = argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (_[ix], gmeans[ix]))
# fpr, tpr, thresholds = roc_curve(y_test[:,i], proba1[:,i])
plt.xlabel("false positive rate")
plt.ylabel("true positive rate")
plt.legend(loc="best")
plt.title("ROC curve")
plt.show()



In [ ]:
cm = confusion_matrix(y_test_one.argmax(axis=1), clf.predict(X_test_ovr).argmax(axis=1))
print("Precision = {}".format(precision_score(y_test_one.argmax(axis=1), clf.predict(X_test_ovr).argmax(axis=1), average='macro')))

print("Recall = {}".format(recall_score(y_test_one.argmax(axis=1), clf.predict(X_test_ovr).argmax(axis=1), average='macro')))

#print("Accuracy = {}".format(accuracy_score(y_test_one.argmax(axis=1), clf.predict(X_test).argmax(axis=1), average='macro')))
cm

In [ ]:
cm = confusion_matrix(y_val_one.argmax(axis=1), clf.predict(x_val_ovr).argmax(axis=1))
print("Precision = {}".format(precision_score(y_val_one.argmax(axis=1), clf.predict(x_val_ovr).argmax(axis=1), average='macro')))

print("Recall = {}".format(recall_score(y_val_one.argmax(axis=1), clf.predict(x_val_ovr).argmax(axis=1), average='macro')))

#print("Accuracy = {}".format(accuracy_score(y_test_one.argmax(axis=1), clf.predict(X_test).argmax(axis=1), average='macro')))
cm

In [ ]:
cm = confusion_matrix(y_tp_one.argmax(axis=1), clf.predict(x_tp).argmax(axis=1))
print("Precision = {}".format(precision_score(y_tp_one.argmax(axis=1), clf.predict(x_tp).argmax(axis=1), average='macro')))

print("Recall = {}".format(recall_score(y_tp_one.argmax(axis=1), clf.predict(x_tp).argmax(axis=1), average='macro')))

#print("Accuracy = {}".format(accuracy_score(y_test_one.argmax(axis=1), clf.predict(X_test).argmax(axis=1), average='macro')))
cm

ONE VS ONE CLASSIFIER for three tags

In [ ]:
X_train_ovo=X_3_tags_train.to_numpy()
X_test_ovo=X_3_tags_test.to_numpy()
y_train_ovo=Y_3_tags_train.to_numpy()
y_test_ovo=Y_3_tags_test.to_numpy()
x_val_ovo=x_val.to_numpy()
y_val_ovo=y_val.to_numpy()


In [ ]:
clf_ovo = OneVsOneClassifier(XGBClassifier(n_estimators=50, max_depth=3,random_state=0))
clf_ovo.fit(X_train_ovo, y_train_ovo)
y_pred_ovo=clf_ovo.predict(X_test_ovo)



In [ ]:
print("Precision = {}".format(precision_score(y_test_ovo, y_pred_ovo, average='macro')))
print("Recall = {}".format(recall_score(y_test_ovo, y_pred_ovo, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test_ovo, y_pred_ovo)))
#auc = roc_auc_score(Y_test, y_pred)
#print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test_ovo,y_pred_ovo)
print(cm)

In [ ]:
y_val_pred_ovo=clf_ovo.predict(x_val_ovo)
print("Precision = {}".format(precision_score(y_val_ovo, y_val_pred_ovo, average='macro')))
print("Recall = {}".format(recall_score(y_val_ovo, y_val_pred_ovo, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_val_ovo, y_val_pred_ovo)))
#auc = roc_auc_score(Y_test, y_pred)
#print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_val_ovo,y_val_pred_ovo)
print(cm)

In [ ]:
y_tp_pred_ovo=clf_ovo.predict(x_tp)
print("Precision = {}".format(precision_score(y_tp, y_tp_pred_ovo, average='macro')))
print("Recall = {}".format(recall_score(y_tp, y_tp_pred_ovo, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_tp, y_tp_pred_ovo)))
#auc = roc_auc_score(Y_test, y_pred)
#print('AUC: %.3f' % auc)
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_tp,y_tp_pred_ovo)
print(cm)